In [1]:
import json

with open('shortest_result.json', 'r', encoding='utf-8') as f:
    datas=json.load(f)

In [2]:
def read_json(file_name):
    # 存放所有字典的列表
    dict_list = []

    # 打开文件并逐行读取
    with open(file_name, 'r', encoding='utf-8') as file:
        for line in file:
            # 去掉行末的换行符（如果有）
            line = line.strip()
            if line:
                # 将每行的字符串解析为字典，并添加到列表中
                dict_list.append(json.loads(line))
    return dict_list

gt = read_json('shortest_test.json')

In [3]:
print(len(datas))
print(len(gt))
print(gt[0])

400
400
{'index': 0, 'input_prompt': 'Find the shortest path between two nodes in an undirected graph. In an undirected graph, (i,j,k) means that node i and node j are connected with an undirected edge with weight k. Given a graph and a pair of nodes, you need to output the shortest path between the two nodes. Q: The nodes are numbered from 0 to 4, and the edges are: (1,4,1) (2,3,6) (3,4,9). Give the weight of the shortest path from node 1 to node 2.', 'answer': '### 16', 'node_range': [2, 20], 'edge_range': [0, 95]}


In [4]:
import re
import io
import sys


In [5]:
id = 180
print(gt[id]['input_prompt'])
print(datas[id]['code'])
print(gt[id]['answer'])

Find the shortest path between two nodes in an undirected graph. In an undirected graph, (i,j,k) means that node i and node j are connected with an undirected edge with weight k. Given a graph and a pair of nodes, you need to output the shortest path between the two nodes. Q: The nodes are numbered from 0 to 51, and the edges are: (0,3,9) (0,5,5) (0,19,1) (1,45,6) (1,42,7) (1,3,6) (2,13,5) (2,50,7) (2,11,9) (2,25,3) (2,39,1) (2,38,8) (3,7,5) (4,43,9) (4,45,7) (4,49,2) (5,10,5) (5,26,6) (5,41,6) (5,38,2) (6,34,4) (6,12,10) (6,47,4) (7,36,10) (7,39,5) (8,27,8) (9,46,5) (9,12,2) (10,18,3) (10,39,9) (10,33,8) (11,16,4) (12,44,8) (12,50,2) (12,24,10) (12,17,1) (13,20,9) (13,25,9) (13,24,1) (14,26,8) (15,34,1) (15,17,10) (15,24,10) (16,23,2) (16,44,4) (17,28,7) (17,42,1) (18,45,4) (19,22,9) (19,32,4) (19,26,7) (19,21,6) (19,31,4) (20,26,5) (20,29,6) (22,34,6) (22,27,9) (23,36,10) (23,29,6) (23,25,5) (24,40,7) (24,27,3) (25,50,2) (26,40,1) (27,48,8) (27,43,2) (28,29,2) (29,40,2) (30,34,8) (32

In [6]:
import networkx as nx

# Create an undirected graph
G = nx.Graph()

# Add edges with weights
edges = [
    (0, 15, 10), (0, 3, 2), (0, 2, 2), (0, 10, 1), (0, 13, 5), (0, 9, 5),
    (1, 5, 10), (1, 4, 4), (1, 9, 2), (1, 7, 2), (1, 14, 9), (1, 3, 1), (1, 12, 4), (1, 10, 7),
    (2, 3, 6), (2, 15, 2), (2, 7, 1), (2, 4, 4), (2, 10, 7),
    (3, 11, 10), (3, 14, 8), (3, 12, 4), (3, 10, 3), (3, 4, 7),
    (4, 11, 4), (4, 6, 2), (4, 5, 4), (4, 15, 3), (4, 12, 6),
    (5, 13, 2), (5, 7, 3), (5, 10, 4), (5, 12, 7), (5, 8, 3),
    (6, 15, 10), (6, 10, 6), (6, 9, 6), (6, 13, 2),
    (7, 9, 5), (7, 13, 5), (7, 11, 3), (7, 12, 2),
    (8, 12, 6), (8, 13, 4), (8, 14, 3),
    (9, 13, 3), (9, 11, 8), (9, 12, 2), (9, 10, 10), (9, 14, 2),
    (10, 13, 6), (10, 11, 7),
    (11, 15, 10), (11, 13, 6), (11, 14, 4),
    (12, 13, 2), (12, 14, 6), (12, 15, 5),
    (13, 15, 9)
]

G.add_weighted_edges_from(edges)

# Find the shortest path from node 12 to node 2
shortest_path = nx.shortest_path(G, source=12, target=2, weight='weight')

# Output the weight of the shortest path
print(nx.shortest_path_length(G, source=12, target=2, weight='weight'))

3


In [7]:
error_cnt = 0
correct_cnt = 0
for i, data in enumerate(datas):
    text = data['code']
    code = re.sub(r'^```python\n|```$', '', text, flags=re.MULTILINE)
    ans = gt[i]['answer']

    output = io.StringIO()

    old_stdout = sys.stdout
    sys.stdout = output

    try:
        exec(code, globals(), globals())
    except:
        sys.stdout = old_stdout
        print(code)
        error_cnt = error_cnt + 1
        continue
    finally:
        sys.stdout = old_stdout
    

    output_content = output.getvalue()
    output_content = str(output_content).strip()
    # print("Captured Output:")
    # print(output_content)
    
    if "the shortest path:" in output_content:
        output_content = output_content.split("the shortest path:")[1].strip()
    
    
    # print(f'llm={output_content} gt={ans}')
        
    if output_content.strip() == str(ans).split('###')[1].strip():
        correct_cnt = correct_cnt + 1
    else:
        print(output_content,ans)

In [8]:
print(error_cnt)
print(correct_cnt)
print(correct_cnt/len(datas))

0
400
1.0
